In [45]:
import pandas as pd
import numpy as np
import os

from combat.pycombat import pycombat

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
from time import time

In [46]:
def import_csv_files(path, starts_with, ends_with = ".csv"):
    file_list = []
    for filename in os.listdir(path):
        if filename.startswith(starts_with) and filename.endswith(ends_with):
            file_path = os.path.join(path, filename)
            file_list.append(pd.read_csv(file_path))
    return file_list

In [47]:
def same_shape(dfs : list):
    return all(df.shape == dfs[0].shape for df in dfs)

In [48]:
def get_average_from_m_datasets(dfs : list):
    if not same_shape:
        print("Dataframes must have the same shape")
    joined = pd.concat(dfs).reset_index()
    return joined.groupby('index').mean()

In [124]:
df_zero_impute = pd.read_csv("./Data/data files/Lumbar_ToImpute.csv")
df_zero_impute.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
df_zero_impute.fillna(0, inplace=True)
df_zero_impute

,P02768,P02787,P01009,P01024,P0C0L5,P02649,P0DOY2,P00738,P06396,P02647,...,Q9NVA2,Q08257,Q56P03,Q9UJ90,P31948,P05107,O95196,O75339,O14960,Q07812
0,-0.072897,0.181743,0.402426,0.184709,0.701173,-0.137677,-0.193675,0.659157,0.227605,-0.337082,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.768857,0.558457,0.379343,0.441253,0.288571,0.000000,-0.524307,2.072155,0.279783,0.679007,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,-0.028010,-0.133621,-0.309150,-0.044071,0.438521,-0.164236,0.063932,0.849340,0.121026,-0.203564,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.377361,0.403682,0.216778,0.722584,0.270382,-0.243587,0.327541,1.936260,-0.069995,1.361930,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,-0.081855,0.075107,0.072067,0.104687,-0.154194,0.247885,-0.479686,-0.306355,0.169838,-0.299773,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,-0.764546,-0.285425,0.000000,-0.390569,0.105446,0.218675,-0.974621,-3.342641,-0.118785,0.000000,...,0.243559,-0.058127,1.359753,-0.065500,-0.440303,0.124877,0.0,-1.110356,-0.137992,0.201295
81,0.470527,0.447341,0.000000,0.664881,0.388296,-0.492861,0.414489,0.046847,0.430552,0.518430,...,0.331662,0.208081,0.000000,-0.788529,-0.146522,0.068939,0.0,0.614697,-0.352913,0.000000
82,-0.141435,0.156426,0.007050,-0.124727,0.376917,0.092540,-0.603993,-2.439421,0.233077,-0.024996,...,-0.283738,0.231163,0.000000,-0.249299,0.079554,-0.354898,0.0,-0.319763,0.000000,0.000000
83,0.555840,0.596420,0.787524,0.674778,0.421446,-0.551402,0.381700,0.619562,0.456220,0.841233,...,0.280245,0.402004,0.000000,0.000000,-0.159438,0.071524,0.0,0.208653,0.428462,0.000000


In [49]:
path = "./Data/data files/imputed"
starts_with = "protein_lumbar"
dfs = import_csv_files(path=path, starts_with=starts_with)
X = get_average_from_m_datasets(dfs)

#df_imputed = pd.read_csv("./Data/data files/imputed/df_protein_lumbar_imputed_75_5_etr.csv")
#X = df_imputed.to_numpy()
df_classes = pd.read_csv("./Data/data files/iNPH_data_protein_median.csv", usecols=["Cortical_biopsy_grouping", "CSF_type", "TMT Set"])
df_classes = df_classes[df_classes["CSF_type"] == "L"].reset_index(drop=True)
tmt_set = df_classes["TMT Set"]
y = df_classes["Cortical_biopsy_grouping"]

In [50]:
def run_combat(df, TMT_set_indices):
    """Run ComBat to reduce batch effect on a dataframe.

    :param df: Dataframe that ComBat is runned on
    :param TMT_set_indices: Labels of which TMT batch each row belongs to.
    :return: DF with ComBat applied to it
    """    
    return pycombat(df.T, TMT_set_indices).T

In [51]:
def add_noise(df, n_samples, std = 0.1):
    """Helper function for simple perturbation function. Adds gaussian noise to randomly sampled rows of the df passed as an argument.

    :param df: df filtered on class
    :param n_samples: Number of samples to perform random sampling on
    :param std: Standard deviation. Defaults to 0.1.
    :return: df with added perturbed samples
    """      
    sampled_df = df.sample(n = n_samples, replace = True)
    gaussian_noise = np.random.normal(0, std, size=sampled_df.shape)
    return sampled_df + gaussian_noise

In [52]:
def simple_perturbation(X, y, n_samples_per_class = None, std = 0.1):
    """Performs a simple perturbation by sampling random rows of classes and adds gaussian noise to them

    :param X: Dataframe with samples to get perturbed
    :param y: Class labels
    :param n_samples_per_class: Decides how many samples per class that are going to be sampled, defaults to None. 
    If none -> all classes will get equal weight according to size of current largest class.
    :param std: How much the noise can deviate from the mean (Standard dev.), defaults to 0.1
    :return: A df with the perturbed samples. A list stating which row belongs to which class.
    """    
    classes = y.value_counts()
    if n_samples_per_class is None:
        largest_class = classes.argmax()
        largest_n_samples = classes.pop(largest_class)
    else:
        largest_n_samples = n_samples_per_class
    df_list = []
    y_new_classes = list(y)
    for idx, n_samples in classes.items():
        perturbed = add_noise(X[y == idx], largest_n_samples - n_samples, std=std)
        df_list.append(perturbed)
        y_new_classes = y_new_classes + [idx] * len(perturbed)
    return pd.concat([X] + df_list, axis=0), y_new_classes#, ignore_index=True)

In [53]:
xgboost = XGBClassifier()
xgboost.name = "XGBoost"
lr = LogisticRegression()
lr.name = "LogisticRegression"
rf = RandomForestClassifier()
rf.name = "RandomForest"

In [54]:
xgboost_params = {"min_child_weight": [1, 5, 10],
                  "gamma": [0.5, 1, 1,5, 2, 5],
                  "subsample": [0.6, 0.8, 1.0],
                  "colsample_bytree": [0.6, 0.8, 1.0],
                  "max_depth": [3, 4, 5],
                  "n_estimators": [100, 500, 1000]}
lr_params = [{"penalty": ["l1"],
              "C": np.arange(0.2, 3.1, 0.2),
              "solver": ["saga"],
              "multi_class": ["multinomial"],
              "max_iter": np.arange(1000, 10001, 2000)
             }]
rf_params = {"n_estimators": np.arange(100, 501, 100),
             "criterion": ["gini", "entropy", "log_loss"],
             "max_depth": [None, 10, 20, 40],
             "max_features": ["sqrt", "log2"],
             "min_samples_leaf": [1, 2, 4]}

In [131]:
#models = [xgboost, lr, rf]
#params = [xgboost_params, lr_params, rf_params]

lr_params = [{"penalty": ["l1"],
              "C": np.arange(0.2, 0.5),
              "solver": ["saga"],
              "multi_class": ["multinomial"],
              "max_iter": [1000]}]
rf_params = {"n_estimators": np.arange(100, 501, 100),
             "criterion": ["gini", "entropy", "log_loss"],
             "max_depth": [None, 10, 20, 40],
             "max_features": ["log2"],
             "min_samples_leaf": [1, 2]}
models = [rf]
params = [rf_params]

In [132]:
def pipeline_pt1(X, df_classes, csf_type, n_samples_per_class = 100, do_combat = True, do_std_scaler = True,):
    type = df_classes[df_classes["CSF_type"] == csf_type].reset_index(drop=True)
    y = type["Cortical_biopsy_grouping"]
    tmt_set = type["TMT Set"].values
    if do_combat:
        X = run_combat(X, tmt_set)
    X, y = simple_perturbation(X, y, n_samples_per_class = n_samples_per_class)
    if do_std_scaler:
        std_scaler = StandardScaler()
        X = std_scaler.fit_transform(X)
    return X, y

In [133]:
def get_best_params(X, y, models, params, cv = 5, n_jobs = -1):
    best_params = {}
    for model, param in zip(models, params):
        start_time = time()
        print(f"{model.name} started.")
        clf = GridSearchCV(model, param_grid=param, cv=cv, n_jobs=n_jobs, scoring="f1_weighted")
        clf.fit(X, y)
        best_params[model.name] = clf.best_params_
        print(f"{model.name} is done in {time() - start_time} seconds.\n" )
    return best_params
    

In [134]:
def pipeline_pt3(X, y, models, params, tmt_set = None, K = 5, n_samples_per_class = 100):
    X = run_combat(X, tmt_set)
    k_fold = StratifiedKFold(n_splits=K, shuffle=True)
    for i, (train_index, test_index) in enumerate(k_fold.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        X_train, y_train = simple_perturbation(X_train, y_train, n_samples_per_class=n_samples_per_class)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        print(f'Fold: {i}')
        print(f'Train: {train_index}')
        print(f'Test: {test_index}')
        #for model in models:
            #pass
        

In [198]:
k_fold = KFold(n_splits=5, shuffle=True)
k_fold.get_n_splits(X=X, y=y)

5

In [201]:
s_k_fold = StratifiedKFold(n_splits=5, shuffle=True, )

array([ 3, 11, 13, 23, 34, 42, 46, 49, 55, 56, 57, 61, 65, 71, 74, 75, 82])

In [137]:
from sklearn.metrics import f1_score
def pipeline(X, y, models, params, tmt_set = None, K = 5, n_samples_per_class = 100):
    X = run_combat(X, tmt_set)
    k_fold = StratifiedKFold(n_splits=K, shuffle=True)
    for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        X_train, y_train = simple_perturbation(X_train, y_train, n_samples_per_class=n_samples_per_class)
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        best_params = get_best_params(X_train, y_train, models, params)
        for model in models:
            best_model = model.__class__(**best_params[model.name])
            best_model.fit(X_train, y_train)
            y_pred = best_model.predict(X_test)
            print(f1_score(y_test, y_pred, average="weighted"))
            

In [138]:
pipeline(X, y, models, params, tmt_set)

Found 15 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
RandomForest started.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/combat/pycombat.py:159: RuntimeWarning: divide by zero encountered in divide
  np.absolute(d_new-d_old)/d_old))  # maximum difference between new and old estimate


RandomForest is done in 31.84743595123291 seconds.

0.33137254901960783
RandomForest started.
RandomForest is done in 32.050607204437256 seconds.

0.17337461300309598
RandomForest started.


KeyboardInterrupt: 